In [33]:
import pandas as pd
import numpy as np

Files in the folder

In [2]:
!ls

data_US_Qinglu.ipynb  data_WHO_Qinglu.ipynb  my_free_doc.txt
data_US_shusma.ipynb  fda_database_doc.txt   Products.txt


In [3]:
import sys
print("Python version: ", sys.version)

Python version:  3.8.10 (default, Jun  4 2021, 15:09:15) 
[GCC 7.5.0]


In [4]:
lst = set()
formulation_set = set()
avoided_formulation = ["inject", "im-iv", "intravenous", "intra-articular", "intramuscular"]

def isAcceptedDrugFormulation (formulation):
    for avoided_form in avoided_formulation:
        if formulation.find(avoided_form) >= 0:
            return False
    return True

with open ("Products.txt", "r") as f:
    f.readline()
    c = 0
    for line in f:
        line_lst = line.split("\t")
        formulation = line_lst[2].lower()
        drugname = line_lst[5].lower()
        if isAcceptedDrugFormulation(formulation):
            lst.add(drugname)
            formulation_set.add(formulation)
        c += 1
lst = list(lst)
print("total lines in the file: ", c)
print("Sample of drug names: ", list(lst)[:10])
print("number of drugs: ", len(lst))
# print("Sample of included formulation: ", list(formulation_set)

total lines in the file:  44873
Sample of drug names:  ['rytary', 'tenofovir alafenamide fumarate', 'serzone', 'butenafine hydrochloride', 'numorphan', 'hydromox', 'pioglitazone', 'betapace af', 'rezipas', 'sectral']
number of drugs:  5583


Generate numpy array of drugs

Provide list of drug names in numpy in case my teammate needs. 

In [5]:
drugset_np = np.array(list(lst))
drugset_np.shape

(5583,)

In [6]:
print(len(lst))

5583


Feature generation step
four types of error
1. omission (missing one letter)
2. transpose (order of two neighbor letters are incorrect)
3. replacement
a. vow-to-vow replacement
b. consonant-to-consonant replacement
4. insertion (insert random letter)



In [7]:
# omission 1 character

def omission(drug, i):
    if i == 0:
        return drug[i+1:]
    elif i == len(drug)-1:
        return drug[:i]
    else:
        return drug[:i] + drug[i+1:]

In [8]:
drug = "abcdef"
for i in range(len(drug)):
    print(omission (drug, i))

bcdef
acdef
abdef
abcef
abcdf
abcde


In [9]:
def transpose(drug, i):
    if i >= len(drug)-1-1:
        i = len(drug)-2
        return drug[:i] + drug[i+1] +drug[i]
    else:
        return drug[:i] + drug[i+1] +drug[i] + drug[i+2:]
        


In [10]:
drug = "abcdef"
for i in range(len(drug)):
    print(i)
    print(transpose(drug, i))

0
bacdef
1
acbdef
2
abdcef
3
abcedf
4
abcdfe
5
abcdfe


In [11]:
import random
VOWELS = list("aeiou")
CONSONANTS = list("bcdfghjklmnpqrstvwxyz")
ALL_LETTERS = VOWELS + CONSONANTS

print(VOWELS)
print(CONSONANTS)
print(ALL_LETTERS)
def replace(drug, i):
    c = drug[i].lower()
    if c in VOWELS:
        r = random.choice(VOWELS)
    elif c in CONSONANTS:
        r = random.choice(CONSONANTS)
    else:
        r = random.choice(ALL_LETTERS)
    ans = drug[:i] + r + drug[i+1:]
    if ans == drug:
        replace(drug, i)
    return ans

['a', 'e', 'i', 'o', 'u']
['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
['a', 'e', 'i', 'o', 'u', 'b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']


In [12]:
drug = "abc__def"
for i in range(len(drug)):
    print(i)
    print(replace(drug, i))

0
abc__def
1
apc__def
2
abb__def
3
abcm_def
4
abc_ndef
5
abc__jef
6
abc__daf
7
abc__def


In [13]:
import random
ALL_LETTERS = VOWELS + CONSONANTS

print(ALL_LETTERS)

def insertion(drug, i):
    r = random.choice(ALL_LETTERS)
    return drug[:i] + r + drug[i:]

['a', 'e', 'i', 'o', 'u', 'b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']


In [14]:
drug = "abcdef"
for i in range(len(drug)+1):
    print(insertion(drug, i))

sabcdef
albcdef
abhcdef
abcgdef
abcdjef
abcdehf
abcdefe


In [15]:
# this feature can be used for unsupervised learning and transformer data intake
def genUnsprvFeatPair(drug):
    temp = [omission(drug, i) for i in range(len(drug))]
    temp += [transpose(drug, i) for i in range(len(drug)-1)]
    temp += [replace(drug, i) for i in range(len(drug))]
    temp += [insertion(drug, i) for i in range(len(drug))]
    return [(x, drug) for x in temp]
    

In [16]:
drug = "drug name"
print(genUnsprvFeatPair(drug))

[('rug name', 'drug name'), ('dug name', 'drug name'), ('drg name', 'drug name'), ('dru name', 'drug name'), ('drugname', 'drug name'), ('drug ame', 'drug name'), ('drug nme', 'drug name'), ('drug nae', 'drug name'), ('drug nam', 'drug name'), ('rdug name', 'drug name'), ('durg name', 'drug name'), ('drgu name', 'drug name'), ('dru gname', 'drug name'), ('drugn ame', 'drug name'), ('drug anme', 'drug name'), ('drug nmae', 'drug name'), ('drug naem', 'drug name'), ('qrug name', 'drug name'), ('dhug name', 'drug name'), ('dreg name', 'drug name'), ('drup name', 'drug name'), ('drugpname', 'drug name'), ('drug qame', 'drug name'), ('drug nume', 'drug name'), ('drug name', 'drug name'), ('drug namo', 'drug name'), ('fdrug name', 'drug name'), ('dlrug name', 'drug name'), ('dreug name', 'drug name'), ('druzg name', 'drug name'), ('drugm name', 'drug name'), ('drug pname', 'drug name'), ('drug ngame', 'drug name'), ('drug napme', 'drug name'), ('drug nampe', 'drug name')]


reference: dictionary spreader: https://sparrow.dev/object-spread-operator-python/

In [17]:
# this feature can be used for NLTK ML models (naive bayes, sklearn models)
def genSprvFeatPair(drug):
    temp = {f"omission_{i}":omission(drug, i) for i in range(len(drug))}
    temp2 = {f"transpose_{i}": transpose(drug, i) for i in range(len(drug)-1)}
    temp3 = {f"replace_{i}": replace(drug, i) for i in range(len(drug))}
    temp4 = {f"insertion_{i}":insertion(drug, i) for i in range(len(drug))}
    return [{**temp, **temp2, **temp3, **temp4}, drug]


In [18]:
#test
drug = "drug"
print(genSprvFeatPair(drug))

[{'omission_0': 'rug', 'omission_1': 'dug', 'omission_2': 'drg', 'omission_3': 'dru', 'transpose_0': 'rdug', 'transpose_1': 'durg', 'transpose_2': 'drgu', 'replace_0': 'prug', 'replace_1': 'dcug', 'replace_2': 'drug', 'replace_3': 'druf', 'insertion_0': 'bdrug', 'insertion_1': 'dgrug', 'insertion_2': 'drqug', 'insertion_3': 'druig'}, 'drug']


Total number of drug names

In [19]:
len(lst)

5583

In [20]:
# unsprv_dataset only first 100. convert the whole dataset would cause too much time? 
Unsprv_dataset = sum([genUnsprvFeatPair(drug) for drug in lst[:]], [])
print(Unsprv_dataset[:10])
print(len(Unsprv_dataset))

[('oprelto', 'goprelto'), ('gprelto', 'goprelto'), ('gorelto', 'goprelto'), ('gopelto', 'goprelto'), ('goprlto', 'goprelto'), ('gopreto', 'goprelto'), ('goprelo', 'goprelto'), ('goprelt', 'goprelto'), ('ogprelto', 'goprelto'), ('gporelto', 'goprelto')]
294741


In [21]:
supervised_dataset = [genSprvFeatPair(drug) for drug in lst]
print(supervised_dataset[:2])
print(len(supervised_dataset))

[[{'omission_0': 'oprelto', 'omission_1': 'gprelto', 'omission_2': 'gorelto', 'omission_3': 'gopelto', 'omission_4': 'goprlto', 'omission_5': 'gopreto', 'omission_6': 'goprelo', 'omission_7': 'goprelt', 'transpose_0': 'ogprelto', 'transpose_1': 'gporelto', 'transpose_2': 'gorpelto', 'transpose_3': 'goperlto', 'transpose_4': 'goprleto', 'transpose_5': 'gopretlo', 'transpose_6': 'goprelot', 'replace_0': 'zoprelto', 'replace_1': 'guprelto', 'replace_2': 'gohrelto', 'replace_3': 'goptelto', 'replace_4': 'goprilto', 'replace_5': 'gopresto', 'replace_6': 'goprelyo', 'replace_7': 'goprelta', 'insertion_0': 'ygoprelto', 'insertion_1': 'geoprelto', 'insertion_2': 'gokprelto', 'insertion_3': 'gopsrelto', 'insertion_4': 'goprqelto', 'insertion_5': 'gopreklto', 'insertion_6': 'goprelxto', 'insertion_7': 'gopreltgo'}, 'goprelto'], [{'omission_0': 'ylet', 'omission_1': 'zlet', 'omission_2': 'zyet', 'omission_3': 'zylt', 'omission_4': 'zyle', 'transpose_0': 'yzlet', 'transpose_1': 'zlyet', 'transpose

# This concludes the data preparation phase.

# Data analysis section
1. lst (measure distance)
2. Unsupervised dataset (transformer)
3. Supervised dataset (for NLTK ML package)


======================================================================


Unsupervised testing
1. Jaccard distance
2. Edit distance
3. Word2Vec (most similar)


Supervised Machine Learning
1. Naive bayes (NLTK)
2. Linear regression (NLTK)
3. Logistic regression (NLTK)
4. Transformer



Unsupervised learning

1. Jaccard distance

reference: https://www.geeksforgeeks.org/correcting-words-using-nltk-in-python/

In [22]:
import nltk
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams

In [23]:
def WordLenDiff(w1, w2):
    return abs(len(w1) - len(w2))

print(WordLenDiff("abc", "abcdef"))
print(WordLenDiff("abcde", "abc"))

3
2


In [24]:
print("Total unsupervised dataset length: ", len(Unsprv_dataset))


Total unsupervised dataset length:  294741


In [29]:
%%time
import random
random.shuffle(Unsprv_dataset)

incorrect_words=[x[0] for x in Unsprv_dataset[:10000]]
ground_truth_words=[x[1] for x in Unsprv_dataset[:10000]]
# print("incorrect_words", incorrect_words)
# print("correct_words", correct_words2)
correct_words = lst

# loop for finding correct spellings
# based on jaccard distance
# and printing the correct word

n_acc = 0
n_total = 0

for i, word in enumerate(incorrect_words):
    if i % 1000 == 0:
        print("i: ", i)
    temp = [(jaccard_distance(set(ngrams(word, 2)),
                              set(ngrams(w, 2))),w)
            for w in correct_words if WordLenDiff(w, word) <=3]
    pred = sorted(temp, key = lambda val:val[0])[0][1]
    n_total += 1
    if pred == ground_truth_words[i]:
        n_acc += 1
print("i final: ", i)
print(f"Jaccard distance accuracy: {n_acc/n_total:6.2f}")

i:  0
i:  1000
i:  2000
i:  3000
i:  4000
i:  5000
i:  6000
i:  7000
i:  8000
i:  9000
i final:  9999
Jaccard distance accuracy:   0.96
CPU times: user 1min 38s, sys: 30.4 ms, total: 1min 38s
Wall time: 1min 38s


For jaccard distance, the full dataset was tested against the method and reached accuracy of 0.96%. 

2. edit distance 
https://www.geeksforgeeks.org/correcting-words-using-nltk-in-python/

In [30]:
from nltk.metrics.distance  import edit_distance

In [43]:
%%time
import random
arr_lst = []
for iter in range(5):
    random.shuffle(Unsprv_dataset)
    incorrect_words=[x[0] for x in Unsprv_dataset]
    ground_truth_words=[x[1] for x in Unsprv_dataset]
    # print("incorrect_words", incorrect_words)
    # print("correct_words", correct_words2)
    correct_words = lst

    # loop for finding correct spellings
    # based on jaccard distance
    # and printing the correct word

    n_acc = 0
    n_total = 0

    for i, word in enumerate(incorrect_words):
        if i % 100 == 0:
            print("i: ", i)                 
        temp = [(edit_distance(word, w),w) for w in correct_words if WordLenDiff(w, word) <=3]
        pred = sorted(temp, key = lambda val:val[0])[0][1]
#         print(word)
#         print(pred)
#         print(ground_truth_words[i])
        n_total += 1
        if pred == ground_truth_words[i]:
            n_acc += 1
        if i>=500:
            break
    acc = n_acc/n_total
    arr_lst.append(acc)
    print(f"{iter}th iter: accuracy: {acc}")
                 
print("i final: ", i)
print(f"Edit distance accuracy: {sum(arr_lst)/len(arr_lst):6.2f}")

i:  0
sulfamethoxazole and trimethoprim and phenazopyridine hydrohcloride
sulfamethoxazole and trimethoprim and phenazopyridine hydrochloride
sulfamethoxazole and trimethoprim and phenazopyridine hydrochloride
reeglan
reglan
reglan
penicillin-x2
penicillin-2
penicillin-2
uqasense
quasense
quasense
methdylin
methylin
methylin
delflex w/ dextrose 2.5% low magnesium low cacium in plastic container
delflex w/ dextrose 2.5% low magnesium low calcium in plastic container
delflex w/ dextrose 2.5% low magnesium low calcium in plastic container
chlorothiazide
chlorothiazide
chlorothiazide
loxapine sucinate
loxapine succinate
loxapine succinate
diflorasone iacetate
diflorasone diacetate
diflorasone diacetate
rfiampin and isoniazid
rifampin and isoniazid
rifampin and isoniazid
0th iter: accuracy: 1.0
i:  0
bacitracin zinc andupolymyxin b sulfate
bacitracin zinc and polymyxin b sulfate
bacitracin zinc and polymyxin b sulfate
reserpine and hydrochlorothiazide-50
reserpine and hydrochlorothiazide-50

In [34]:
print(f"Edit distance accuracy: {sum(arr_lst)/len(arr_lst):8.4f}")

Edit distance accuracy:   0.9900


Given the larger time consumed for each search in edit distance, instead that iterating total 290000 samples, we randomly pick 100 samples from the full list and test accuracy. We iterate the process for 5 times and average the accuracy. 

3. word2vec
- reference to hw2

In [24]:
from gensim.models import Word2Vec

word2vec accuracy is 0. likely because each drug name only occurs 1 time. 

In [25]:
%%time
import random

acc_w2v_lst = []
for iter in range(5):
    random.shuffle(Unsprv_dataset)
    incorrect_words=[x[0] for x in Unsprv_dataset[:100]]
    ground_truth_words=[x[1] for x in Unsprv_dataset[:100]]

    n_acc = 0
    n_total = 0
    for i, w_incorrect in enumerate(incorrect_words):
        if i % 10 == 0:
            print("i: ", i)
        w_correct = ground_truth_words[i]
        lst_new = [w_incorrect] +lst
        model = Word2Vec(sentences=[lst_new], min_count=1, workers=4, iter=10)
        output = model.wv.most_similar(w_incorrect)[0][0]
        n_total += 1
        if w_correct == output:
            n_acc+=1
    print("number of matched: ", n_acc)
    print(f"{iter}th iter accuracy: {n_acc/n_total}")
    acc_w2v_lst.append(n_acc/n_total)

i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
0th iter accuracy: 0.0
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
1th iter accuracy: 0.0
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  1
2th iter accuracy: 0.01
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
3th iter accuracy: 0.0
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
4th iter accuracy: 0.0
CPU times: user 12min 52s, sys: 3min 34s, total: 16min 26s
Wall time: 9min 52s


In [26]:
print(f"word2vector accuracy for best match: {sum(acc_w2v_lst)/len(acc_w2v_lst):8.4f}")

word2vector accuracy for best match:   0.0020


In [27]:
%%time
import random

acc_w2v_lst = []
for iter in range(5):
    random.shuffle(Unsprv_dataset)
    incorrect_words=[x[0] for x in Unsprv_dataset[:100]]
    ground_truth_words=[x[1] for x in Unsprv_dataset[:100]]

    n_acc = 0
    n_total = 0
    for i, w_incorrect in enumerate(incorrect_words):
        if i % 10 == 0:
            print("i: ", i)
        w_correct = ground_truth_words[i]
        lst_new = [w_incorrect] +lst
        model = Word2Vec(sentences=[lst_new], min_count=1, workers=4, iter=10)
        output = model.wv.most_similar(w_incorrect)[:5]
        output = [x[0] for x in output]
        n_total += 1
        if w_correct in output:
            n_acc+=1
    print("number of matched: ", n_acc)
    print(f"{iter}th iter accuracy: {n_acc/n_total}")
    acc_w2v_lst.append(n_acc/n_total)

i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
0th iter accuracy: 0.0
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
1th iter accuracy: 0.0
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
2th iter accuracy: 0.0
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
3th iter accuracy: 0.0
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
number of matched:  0
4th iter accuracy: 0.0
CPU times: user 12min 50s, sys: 3min 31s, total: 16min 21s
Wall time: 9min 55s


In [30]:
print(f"word2vec accuracy for top 5: {sum(acc_w2v_lst)/len(acc_w2v_lst):8.4f}")

word2vec accuracy for top 5:   0.0000


Word2Vec is not right tool to solve spelling error check issue. 

Supervised machine learning models

In [40]:
import nltk

In [44]:
%%time

# up to 1st 1000 drugs 
import random
random.shuffle(supervised_dataset)

dataset = supervised_dataset[:1000]

split = 0.80
size = int(len(dataset)*split)
print("size: ", size)

print("length of features: ", len(dataset))
train_set, test_set = dataset[:size], dataset[size:]

classifier = nltk.NaiveBayesClassifier.train(train_set)
print("Train on train set and Accuracy using test set: ")
print(nltk.classify.accuracy(classifier, test_set))
print("Accuracy using train set (this is against protocol in machine learning. train on train set and test on train set): ")
print(nltk.classify.accuracy(classifier, train_set))

size:  800
length of features:  1000
Train on train set and Accuracy using test set: 
0.0
Accuracy using train set (this is against protocol in machine learning. train on train set and test on train set): 
1.0
CPU times: user 1min, sys: 46.7 ms, total: 1min
Wall time: 1min


The above result can be explained by scarcity of drug spelling. in order to reach 100% match in drug names, ML model can't retrieve answer from training set because spelling is very distinct. Each drug name is in its own label (class). 

Alternatively, since drug name is limited vocabulary, all wrong spelling are saved, we can store all drug mis-spelling in the model parameters. It will product very good result given taht the model sees this mis-spelling in the past. Therefore, Model can be built from previous failures (misspellings).  

In [45]:
# !pip install pyspellchecker

# PySpellChecker documentation

https://pypi.org/project/pyspellchecker/

from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(['something', 'is', 'hapenning', 'here'])

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

If the Word Frequency list is not to your liking, you can add additional text to generate a more appropriate list for your use case.


from spellchecker import SpellChecker

spell = SpellChecker()  # loads default word frequency list
spell.word_frequency.load_text_file('./my_free_text_doc.txt')




In [46]:
from spellchecker import SpellChecker

In [47]:
spell = SpellChecker()

In [48]:
incorrect_words=[x[0] for x in Unsprv_dataset[:100]]
ground_truth_words=[x[1] for x in Unsprv_dataset[:100]]
w1 = incorrect_words[0]
w2 = ground_truth_words[0]
print(w1, w2)

anafwanil anafranil


In [56]:
%%time
spell.distance = 1
print(len(incorrect_words))
mispelled = spell.unknown(incorrect_words)
n_acc = 0
n_total = 0
for i, word in enumerate(mispelled):
    n_total += 1
    if i % 10 == 0:
        print("i: ", i)
    corrected = spell.correction(word)
    if corrected == ground_truth_words[i]:
        print(corrected)
        n_acc += 1
#     if spell.correction(word) is not None:
#         print(spell.correction(word))
print("if there is no output, means all 100 are incorrected, accurarcy = 0.")        
print("end")

500
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
i:  100
i:  110
i:  120
i:  130
i:  140
i:  150
i:  160
i:  170
i:  180
i:  190
i:  200
i:  210
i:  220
i:  230
i:  240
i:  250
i:  260
i:  270
i:  280
i:  290
i:  300
i:  310
i:  320
i:  330
i:  340
i:  350
i:  360
i:  370
i:  380
i:  390
i:  400
i:  410
i:  420
i:  430
i:  440
i:  450
i:  460
i:  470
if there is no output, means all 100 are incorrected, accurarcy = 0.
end
CPU times: user 969 ms, sys: 3.03 ms, total: 972 ms
Wall time: 966 ms


In [57]:
print(f"accuracy: {n_acc/n_total} ")

accuracy: 0.0 


If we used pre-loaded database in the SpellChecker, the accuracy would be very low(acc = 0) because pre-loaded database is not tailored to clinical field. 

Add drug name dictionary to pyspellchecker package. 

In [61]:
print("drug list:", len(lst))

drug list: 5583


In [63]:
spell.word_frequency.load_words(lst)
spell.distance = 2

In [ ]:
#to ensure that dataset is correctly stored here
print(len(Unsprv_dataset))

In [64]:
incorrect_words=[x[0] for x in Unsprv_dataset[:500]]
ground_truth_words=[x[1] for x in Unsprv_dataset[:500]]

In [65]:
%%time
print(len(incorrect_words))
mispelled = spell.unknown(incorrect_words)
n_acc = 0
n_count = 0
for i, word in enumerate(mispelled):
    n_count += 1
    corrected = spell.correction(word)
    if corrected == ground_truth_words[i]:
        n_acc +=1
#     if spell.correction(word) is not None:
#         print(spell.correction(word))
print(f"accuracy is {n_acc/n_count: 6.2f}")     
print("end")

500
accuracy is   0.00
end
CPU times: user 948 ms, sys: 1.99 ms, total: 950 ms
Wall time: 948 ms



//sometimes, this part is working. 
If we add drug name to pyspellchecker dictionary, accuracy is 0.12 for 1000 tests and accuracy would drop to lower number. 

500 words: acc: 0.26
1000 words: acc of 0.12
5000 words: acc of 0.03
    

In [12]:
print(lst[:10])
uniword = [w.split()[0] for w in lst]  
print(uniword[:10])
updated_lst = list(set(lst+uniword))
print(updated_lst[:10])
print("updated list: ", len(updated_lst))

['rytary', 'tenofovir alafenamide fumarate', 'serzone', 'butenafine hydrochloride', 'numorphan', 'hydromox', 'pioglitazone', 'betapace af', 'rezipas', 'sectral']
['rytary', 'tenofovir', 'serzone', 'butenafine', 'numorphan', 'hydromox', 'pioglitazone', 'betapace', 'rezipas', 'sectral']
['tenofovir alafenamide fumarate', 'rytary', 'serzone', 'butenafine hydrochloride', 'numorphan', 'hydromox', 'pioglitazone', 'betapace af', 'rezipas', 'sectral']
updated list:  6394


In [13]:
lst = updated_lst

In [14]:
drugset = set(lst)

In [15]:
from nltk.metrics.distance  import edit_distance


import nltk
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams


def WordLenDiff(w1, w2):
    return abs(len(w1) - len(w2))



In [16]:
correct_words = lst
def edit_distance_rec(word):
    # loop for finding correct spellings
    # based on jaccard distance
    # and printing the correct word
    temp = [(edit_distance(word, w),w) for w in correct_words if WordLenDiff(w, word) <=3]
    return sorted(temp, key = lambda val:val[0])[0][1]

print(edit_distance_rec('reeglan'))

reglan


In [17]:
def jaccard_distance_rec(word):
    temp = [(jaccard_distance(set(ngrams(word, 2)),
                              set(ngrams(w, 2))),w)
            for w in correct_words if WordLenDiff(w, word) <=3]
    return sorted(temp, key = lambda val:val[0])[0][1]

print(jaccard_distance_rec('reeglan'))

reglan


In [28]:
%%time

def drug_inquiry(input_drug):
    drugset = set(lst)
    print(f"\nYour entered {input_drug}")

    if input_drug.lower() in drugset:
        print(f"{input_drug} is found in drug dictionary. ")
    else:
        print(f"\n{input_drug} is not spelled correctly")
        print(f"Recommended drug spelling using Edit distance: {edit_distance_rec(input_drug)}")
        print(f"Recommended drug spelling using Jaccard distance: {jaccard_distance_rec(input_drug)}")

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


Demo of simple inquiry system

In [32]:
%%time

while True:
    input_drug = input("\nPlease enter drug name or enter STOP to exit: ")
    if input_drug.lower() == "stop":
        print("\nExit the drug inquiry system. ")
        break;
    else:
        drug_inquiry(input_drug)


Please enter drug name or enter STOP to exit: atorvastatin

Your entered atorvastatin
atorvastatin is found in drug dictionary. 

Please enter drug name or enter STOP to exit: atorvasttin

Your entered atorvasttin

atorvasttin is not spelled correctly
Recommended drug spelling using edit distance: atorvastatin
Recommended drug spelling using jaccard distance: atorvastatin

Please enter drug name or enter STOP to exit: stop

Exit the drug inquiry system. 
CPU times: user 359 ms, sys: 27.5 ms, total: 387 ms
Wall time: 22.5 s


In [36]:
%%time

while True:
    input_drug = input("\nPlease enter drug name or enter STOP to exit: \n")
    if input_drug.lower() == "stop":
        print("\nExit the drug inquiry system. ")
        break;
    else:
        drug_inquiry(input_drug)


Please enter drug name or enter STOP to exit: 
stop

Exit the drug inquiry system. 
CPU times: user 1.07 s, sys: 371 ms, total: 1.44 s
Wall time: 5min 5s


# This concludes the section for Qinglu Ren's part. 